In [13]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [42]:
SCOPES = ['https://www.googleapis.com/auth/calendar']

In [35]:
def get_events_from_calendar():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)
        now = datetime.datetime.now().isoformat() + 'Z'
        events_result = service.events().list(calendarId='primary', timeMin=now, maxResults=10, singleEvents=True, orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')

        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(f'{start} {event["summary"]}')
    except HttpError as e:
        print(f'An error occurred: {e}')

In [36]:
get_events_from_calendar()

2025-02-22T17:00:00-05:00 Workout
2025-02-22T19:00:00-05:00 Intro to Gen AI
2025-02-22T19:00:00-05:00 Business growth
2025-02-23T17:00:00-05:00 Workout
2025-02-23T19:00:00-05:00 Business growth
2025-02-24T17:00:00-05:00 Workout
2025-02-24T19:00:00-05:00 Business growth
2025-02-24T21:00:00-05:00 Akash call
2025-02-25T17:00:00-05:00 Workout
2025-02-25T19:00:00-05:00 Business growth


In [48]:
def add_events_to_calendar():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)
        
        event = {
            "summary": "demo event",
            "location": "online",
            "description": "demo event",
            "colorId" : 1,
            "start" : {
                "dateTime": "2025-02-22T16:00:00",
                "timeZone": "EST"
            },
            "end" : {
                "dateTime": "2025-02-22T17:00:00",
                "timeZone": "EST"
            },
            "recurrence": [
                "RRULE:FREQ=DAILY;COUNT=1"
            ],
            "attendees": [
                {"email": "sathwik238@gmail.com"}
            ]
        }

        event = service.events().insert(calendarId='primary', body=event).execute()
        print(f'Event created: {event.get("htmlLink")}')


    except HttpError as e:
        print(f'An error occurred: {e}')

In [49]:
add_events_to_calendar()

Event created: https://www.google.com/calendar/event?eid=dW9tMXJmMTVpaXJ2NDBzMjAxbWt2YmNobHNfMjAyNTAyMjJUMjEwMDAwWiBzYXRod2lrMjM4QG0
